In [1]:
import os

In [2]:
import google.adk

c:\PG\Projects\Personalized-Finance-Agent\venv\lib\site-packages\google\api_core\_python_version_support.py:266: FutureWarning: You are using a Python version (3.10.11) which Google will stop supporting in new releases of google.api_core once it reaches its end of life (2026-10-04). Please upgrade to the latest Python version, or at least Python 3.11, to continue receiving updates for google.api_core past that date.
  warnings.warn(message, FutureWarning)


In [ ]:
from dotenv import load_dotenv
import os
load_dotenv()
os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")

In [17]:
from google.adk.agents import LlmAgent, SequentialAgent, ParallelAgent, LoopAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from google.adk.tools import AgentTool, FunctionTool, google_search
from google.adk.code_executors import BuiltInCodeExecutor
from google.genai import types

In [24]:
def Http_options():
    retry_config = types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],  # Retry on these HTTP errors
    )
    return retry_config

def show_python_code_and_result(response):
    for i in range(len(response)):
        # Check if the response contains a valid function call result from the code executor
        if (
            (response[i].content.parts)
            and (response[i].content.parts[0])
            and (response[i].content.parts[0].function_response)
            and (response[i].content.parts[0].function_response.response)
        ):
            response_code = response[i].content.parts[0].function_response.response
            if "result" in response_code and response_code["result"] != "```":
                if "tool_code" in response_code["result"]:
                    print(
                        "Generated Python Code >> ",
                        response_code["result"].replace("tool_code", ""),
                    )
                else:
                    print("Generated Python Response >> ", response_code["result"])


In [139]:
class calculator_agent:
    def __init__(self):
        self.agent = LlmAgent(
             name="CalculationAgent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options = Http_options()),
    instruction="""
You are a specialized code-only calculator. You ONLY respond with Python code.

Your job is to translate any calculation request - from simple arithmetic to full financial and complex models - into a single Python script that calculates the required result.

STRICT RULES:
1. Your output MUST be ONLY a Python code block (```python ... ```).
2. You MUST NOT write any text outside the code block.
3. The Python code MUST compute the final result by itself and DO NOT rely on any packages.
4. The Python code MUST print ONLY the final result.
5. You MUST NOT do any math yourself - the Python code must do all calculations.
6. You ARE allowed to generate full functions, classes, helper logic, imports, and multiple lines of code.
7. You MUST NOT add comments inside the code - only pure code.

Your job is to produce executable Python that solves the user’s calculation request.
""",
    code_executor=BuiltInCodeExecutor(),  # Use the built-in Code Executor Tool. This gives the agent code execution capabilities

        )

In [98]:
agent = LlmAgent(
    name="enhanced_agent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=Http_options()),
    # Updated instruction
    instruction="""
You are now a calculator assistant.

RULES:
1. If the task is to find answer for ANY calculation - simple or complex - you MUST call the calculator_agent tool and create the required Python function to calculate and execute it.
2. You MUST pass the user's entire request directly to the tool.
3. You MUST NOT generate any Python code yourself.
4. After the tool runs and executes the code, you MUST return the tool's result to the user.
"""
,
    tools=[
        AgentTool(agent=calculator_agent().agent),  # Using another agent as a tool!
    ],
)

In [83]:
runner = InMemoryRunner(agent=agent)

In [79]:
response = await runner.run_debug(
    "What is the monthly mortgage payment that I have to pay for a $620,000 home, $120,000 downpayment, 5 percent interest, 25 years."
)


 ### Created new session: debug_session_id

User > What is the monthly mortgage payment that I have to pay for a $620,000 home, $120,000 downpayment, 5 percent interest, 25 years.
enhanced_agent > The monthly mortgage payment for the house would be approximately $3,597.58.


In [93]:
response1 = await runner.run_debug(
    "I want a full amortization schedule for a $700,000 mortgage, $100,000 downpayment, 4.2% interest, 30 years. "
)


 ### Continue session: debug_session_id

User > I want a full amortization schedule for a $700,000 mortgage, $100,000 downpayment, 4.2% interest, 30 years. 
enhanced_agent > Here is the amortization schedule for your mortgage:

Month | Payment   | Interest Paid | Principal Paid | Remaining Balance
----------------------------------------------------------------------
1     | 2922.96   | 2100.00       | 822.96         | 599177.04
2     | 2922.96   | 2097.07       | 825.89         | 598351.15
3     | 2922.96   | 2094.12       | 828.84         | 597522.31
4     | 2922.96   | 2091.14       | 831.82         | 596690.49
5     | 2922.96   | 2088.14       | 834.82         | 595855.67
6     | 2922.96   | 2085.12       | 837.84         | 595017.83
7     | 2922.96   | 2082.08       | 840.88         | 594176.95
8     | 2922.96   | 2079.02       | 843.94         | 593333.01
9     | 2922.96   | 2075.95       | 847.01         | 592486.00
10    | 2922.96   | 2072.85       | 850.11         | 591635.89

In [94]:
response1 = await runner.run_debug(
    "Calculate the compound interest for $12,000 at 6% for 10 years."
)


 ### Continue session: debug_session_id

User > Calculate the compound interest for $12,000 at 6% for 10 years.
enhanced_agent > The compound interest is $9490.17.


In [71]:
show_python_code_and_result(response)

Generated Python Response >>  ```python
import numpy_financial as npf

loan_amount = 620000 - 120000
interest_rate = 0.05
loan_term_years = 25
number_of_payments = loan_term_years * 12

monthly_payment = npf.pmt(interest_rate / 12, number_of_payments, -loan_amount)

print(f"{monthly_payment=}")
```
Generated Python Response >>  ```python
import numpy as np

loan_amount = 620000 - 120000
interest_rate = 0.05
loan_term_years = 25
number_of_payments = loan_term_years * 12
monthly_payment = np.pmt(interest_rate / 12, number_of_payments, -loan_amount)
print(monthly_payment)
```


In [80]:
show_python_code_and_result(response)

Generated Python Response >>  ```python
import numpy_financial as npf

home_price = 620000
down_payment = 120000
interest_rate = 0.05
loan_term_years = 25

loan_amount = home_price - down_payment
monthly_interest_rate = interest_rate / 12
loan_term_months = loan_term_years * 12

monthly_payment = npf.pmt(monthly_interest_rate, loan_term_months, -loan_amount)

print(monthly_payment)
```
Generated Python Response >>  The previous request failed because the `numpy` library does not have a `pmt` function. The `pmt` function is typically found in the `numpy_financial` library. To proceed, I will use `numpy_financial`.


The `numpy_financial` library is not installed in the current environment. I will proceed by installing it and then executing the original request.


The installation of `numpy-financial` failed. I will try to run the calculation directly, assuming the library might be available in the environment despite the installation error. If it fails again, it indicates a persistent i

In [87]:
show_python_code_and_result(response1)

Generated Python Response >>  ```python
principal = 620000 - 120000
interest_rate = 0.05
years = 25
 
monthly_interest_rate = interest_rate / 12
number_of_payments = years * 12
 
if monthly_interest_rate > 0:
    monthly_payment = (principal * monthly_interest_rate) / (1 - (1 + monthly_interest_rate)**(-number_of_payments))
else:
    monthly_payment = principal / number_of_payments
 
print(monthly_payment)
```
```
Generated Python Response >>  ```python
principal = 620000 - 120000
interest_rate = 0.05
years = 25
monthly_interest_rate = interest_rate / 12
number_of_payments = years * 12

monthly_payment = (principal * monthly_interest_rate) / (1 - (1 + monthly_interest_rate)**-number_of_payments)

print(f"{monthly_payment=}")
```
Generated Python Code >>  
import sympy

P = 620000 - 120000
r = 0.05 / 12
n = 25 * 12

M = P * (r * (1 + r)**n) / ((1 + r)**n - 1)
print(f'{M=}')

Generated Python Response >>  2922.95


In [89]:
show_python_code_and_result(response1)

Generated Python Response >>  ```python
def calculate_monthly_mortgage(principal, annual_interest_rate, loan_term_years):
    monthly_interest_rate = annual_interest_rate / 12
    number_of_payments = loan_term_years * 12
    
    if monthly_interest_rate == 0:
        monthly_payment = principal / number_of_payments
    else:
        monthly_payment = (principal * monthly_interest_rate) / (1 - (1 + monthly_interest_rate)**(-number_of_payments))
    
    return monthly_payment

home_price = 620000
down_payment = 120000
loan_amount = home_price - down_payment
annual_interest_rate = 0.05
loan_term_years = 25

monthly_payment = calculate_monthly_mortgage(loan_amount, annual_interest_rate, loan_term_years)
print(monthly_payment)
```
```json
{"code_output": "2871.5569859163963"}
```
```python
print(2871.5569859163963)
```


In [95]:
show_python_code_and_result(response1)

Generated Python Response >>  9490.17


In [140]:
class Interest_Finder_Agent:
    def __init__(self):
        self.agent = LlmAgent(
            name = "Interest_Finder",
            model = Gemini(model="gemini-2.5-flash-lite", retry_options = Http_options()),
            instruction= """
            You are an interest-rate extraction agent.

            TASK:
            - Use GoogleSearch to look up the latest REAL mortgage interest rate for any bank in the U.S. or Canada as given.
            - Only return the numeric rate and the bank name (Example format: [5.24, CIBC]).
            - No words. No explanation. No symbols. Only the number and the bank name.

            SEARCH RULES:
            - Prefer official bank websites (.ca, .com).
            - Prefer RateHub (Canada) or Zillow (US) as backup.
            - Ignore ads, loan brokers, or sponsored content.
            """,
            tools= [google_search]
                    )

In [119]:
agent = LlmAgent(
    name="enhanced_agent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=Http_options()),
    # Updated instruction
    instruction="""
You are now an Interest Search assistant.

RULES:
Your responsibilities:

1. If the user asks for a mortgage interest rate:
   - Call InterestSearchAgent tool.
   - Return the numeric rate and Bank name.
2. If the location/country is not given, try looking up and extract the Country name from the given name.
3. If cannot find the country or any countries other than Canada or US is identified, prompt for the country name strictly giving two options: Canada or US.
"""
,
    tools=[
        AgentTool(agent=Interest_Finder_Agent().agent),  # Using another agent as a tool!
    ],
)

In [120]:
runner = InMemoryRunner(agent=agent)
response = await runner.run_debug(
    "What is the RBC 5-year fixed mortgage rate?"
)


 ### Created new session: debug_session_id

User > What is the RBC 5-year fixed mortgage rate?
enhanced_agent > The RBC 5-year fixed mortgage rate is 4.49%.


In [121]:
show_python_code_and_result(response)

Generated Python Response >>  [4.49, RBC]


In [132]:
agent = LlmAgent(
    name="enhanced_agent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=Http_options()),
    # Updated instruction
    instruction="""
You are the Mortgage Orchestration Agent.

Your responsibilities:

1. If the user asks for a mortgage interest rate:
   - Call InterestSearchAgent tool.
   - Return ONLY the numeric rate.

2. If the user asks for a mortgage calculation:
   - Call Calculator_agent tool with the keyword "Mortgage calculator".
   - Pass all user parameters given.
   - Forward EXACTLY the Python execution output to the user.

3. NEVER generate calculations or code yourself.
4. ALWAYS return tool outputs directly without modification.
5. If information is missing (e.g., bank interest rate),
   FIRST call the search agent to obtain the numeric rate,
   THEN call the calculation agent.

You are the coordinator - you decide which agent runs first."""
,
    tools=[
        AgentTool(agent=Interest_Finder_Agent().agent), AgentTool(agent=calculator_agent().agent)  # Using another agent as a tool!
    ],
)

In [124]:
runner = InMemoryRunner(agent=agent)
response = await runner.run_debug(
    "What is the monthly mortgage payment for a $620,000 home, $120,000 downpayment, RBC 5-year fixed mortgage rate, 25 years?"
)


 ### Created new session: debug_session_id

User > What is the monthly mortgage payment for a $620,000 home, $120,000 downpayment, RBC 5-year fixed mortgage rate, 25 years?
enhanced_agent > The monthly mortgage payment is $2,905.61.


In [125]:
show_python_code_and_result(response)

Generated Python Response >>   The 5-year fixed mortgage rate at RBC Royal Bank is 4.590%.
Generated Python Response >>  ```python
def calculate_mortgage_payment(principal, interest_rate, loan_term_years):
    monthly_interest_rate = interest_rate / 100 / 12
    number_of_payments = loan_term_years * 12
    if monthly_interest_rate == 0:
        monthly_payment = principal / number_of_payments
    else:
        monthly_payment = principal * (monthly_interest_rate * (1 + monthly_interest_rate)**number_of_payments) / ((1 + monthly_interest_rate)**number_of_payments - 1)
    return monthly_payment
home_price = 620000
down_payment = 120000
interest_rate = 4.590
loan_term_years = 25
loan_amount = home_price - down_payment
monthly_payment = calculate_mortgage_payment(loan_amount, interest_rate, loan_term_years)
print(f"{monthly_payment:.2f}")
```


In [141]:
class Mortgage_agent:
    def __init__(self):
            self.interest_finder = Interest_Finder_Agent().agent
            self.Calculator = calculator_agent().agent

            self.agent = LlmAgent(
            name="Mortgage_agent",
            model=Gemini(model="gemini-2.5-flash-lite", retry_options=Http_options()),
            # Updated instruction
            instruction="""
        You are the Mortgage Orchestration Agent.

        Your responsibilities:

        1. If the user asks for a mortgage interest rate:
        - Call Interest_finder tool.
        - Return ONLY the numeric rate.

        2. If the user asks for a mortgage calculation:
        - Call Calculator_agent tool with the keyword "Mortgage calculator".
        - Pass all user parameters given.
        - Forward EXACTLY the Python execution output to the user.

        3. NEVER generate calculations or code yourself.
        4. ALWAYS return tool outputs directly without modification.
        5. If information is missing (e.g., bank interest rate),
        FIRST call the Interest_finder agent to obtain the numeric rate,
        THEN call the calculation agent.

        You are the coordinator - you decide which agent runs first."""
        ,
            tools=[
                AgentTool(self.interest_finder), AgentTool(self.Calculator)  # Using another agent as a tool!
            ],
        )

In [137]:
runner = InMemoryRunner(agent=Mortgage_agent().agent)
response = await runner.run_debug(
    "What is the monthly mortgage payment for a $620,000 home, $120,000 downpayment, RBC 5-year fixed mortgage rate, 25 years?"
)


 ### Created new session: debug_session_id

User > What is the monthly mortgage payment for a $620,000 home, $120,000 downpayment, RBC 5-year fixed mortgage rate, 25 years?
Mortgage_agent > The monthly mortgage payment is approximately $2804.77.



In [138]:
show_python_code_and_result(response)

Generated Python Response >>  [4.59, RBC]
Generated Python Response >>  The user is asking for a mortgage payment calculation. I can use the standard formula for calculating the monthly payment of a loan. The formula is:

M = P [ i(1 + i)^n ] / [ (1 + i)^n – 1]

Where:
M = Monthly Payment
P = Principal Loan Amount
i = Monthly Interest Rate
n = Total Number of Payments (Loan Term in Years * 12)

I will need to convert the annual interest rate to a monthly interest rate and the loan term in years to the total number of months.

P = 500000
Annual Interest Rate = 4.59% = 0.0459
Loan Term = 25 years

Monthly Interest Rate (i) = 0.0459 / 12
Total Number of Payments (n) = 25 * 12

I will now implement this in Python.
The monthly mortgage payment is approximately $2804.77.
```python
P = 500000
annual_interest_rate = 0.0459
loan_term_years = 25

i = annual_interest_rate / 12
n = loan_term_years * 12

M = P * (i * (1 + i)**n) / ((1 + i)**n - 1)

print(round(M, 2))
```


In [134]:
show_python_code_and_result(response)

Generated Python Response >>  [4.59, RBC Royal Bank]
Generated Python Response >>  ```python
def calculate_mortgage(principal, interest_rate, loan_term_years, loan_term_months):
    monthly_interest_rate = interest_rate / 100 / 12
    number_of_payments = loan_term_years * 12 + loan_term_months
    if monthly_interest_rate > 0:
        monthly_payment = principal * (monthly_interest_rate * (1 + monthly_interest_rate)**number_of_payments) / ((1 + monthly_interest_rate)**number_of_payments - 1)
    else:
        monthly_payment = principal / number_of_payments
    return monthly_payment
home_price = 620000
down_payment = 120000
loan_amount = home_price - down_payment
interest_rate = 4.59
loan_term_years = 25
loan_term_months = 0
monthly_payment = calculate_mortgage(loan_amount, interest_rate, loan_term_years, loan_term_months)
print(monthly_payment)
```


In [142]:
runner = InMemoryRunner(agent=Mortgage_agent().agent)
response = await runner.run_debug(
    "What is the monthly mortgage payment for a $620,000 home, $120,000 downpayment, RBC 5-year fixed mortgage rate, 25 years?"
)


 ### Created new session: debug_session_id

User > What is the monthly mortgage payment for a $620,000 home, $120,000 downpayment, RBC 5-year fixed mortgage rate, 25 years?
Mortgage_agent > The monthly mortgage payment is $2,906.16.


In [143]:
show_python_code_and_result(response)

Generated Python Response >>  4.59, RBC
Generated Python Response >>  ```python
capital = 500000
down_payment = 120000
interest_rate = 4.59
term_years = 25

loan_amount = capital - down_payment
monthly_interest_rate = (interest_rate / 100) / 12
term_months = term_years * 12

if monthly_interest_rate == 0:
    monthly_payment = loan_amount / term_months
else:
    monthly_payment = loan_amount * (monthly_interest_rate * (1 + monthly_interest_rate)**term_months) / ((1 + monthly_interest_rate)**term_months - 1)

print(f'{monthly_payment=}')

```


In [144]:
show_python_code_and_result(response)

Generated Python Response >>  4.59, RBC
Generated Python Response >>  ```python
capital = 500000
down_payment = 120000
interest_rate = 4.59
term_years = 25

loan_amount = capital - down_payment
monthly_interest_rate = (interest_rate / 100) / 12
term_months = term_years * 12

if monthly_interest_rate == 0:
    monthly_payment = loan_amount / term_months
else:
    monthly_payment = loan_amount * (monthly_interest_rate * (1 + monthly_interest_rate)**term_months) / ((1 + monthly_interest_rate)**term_months - 1)

print(f'{monthly_payment=}')

```


In [239]:
import yfinance as yf

def get_stock_yahoo(symbol: str)-> dict:
    ticker = yf.Ticker(symbol)

    current_price = ticker.info.get("regularMarketPrice")
    history = ticker.history(period="1mo")

    return {
        "symbol": symbol,
        "current_price": current_price,
        "history": history
    }


# Test
result = get_stock_yahoo("AAPL")
print("Current Price:", result["current_price"])
print(result["history"].tail())


Current Price: 271.49
                                 Open        High         Low       Close  \
Date                                                                        
2025-11-17 00:00:00-05:00  268.820007  270.489990  265.730011  267.459991   
2025-11-18 00:00:00-05:00  269.989990  270.709991  265.320007  267.440002   
2025-11-19 00:00:00-05:00  265.529999  272.209991  265.500000  268.559998   
2025-11-20 00:00:00-05:00  270.829987  275.429993  265.920013  266.250000   
2025-11-21 00:00:00-05:00  265.950012  273.329987  265.670013  271.489990   

                             Volume  Dividends  Stock Splits  
Date                                                          
2025-11-17 00:00:00-05:00  45018300        0.0           0.0  
2025-11-18 00:00:00-05:00  45677300        0.0           0.0  
2025-11-19 00:00:00-05:00  40424500        0.0           0.0  
2025-11-20 00:00:00-05:00  45823600        0.0           0.0  
2025-11-21 00:00:00-05:00  58784100        0.0           0.0

In [147]:
# Test
result = get_stock_yahoo("NVDA")
print("Current Price:", result["current_price"])
print(result["history"].tail())

Current Price: 180.64
                                 Open        High         Low       Close  \
Date                                                                        
2025-11-14 00:00:00-05:00  182.860001  191.009995  180.580002  190.169998   
2025-11-17 00:00:00-05:00  185.970001  189.000000  184.320007  186.600006   
2025-11-18 00:00:00-05:00  183.380005  184.800003  179.649994  181.360001   
2025-11-19 00:00:00-05:00  184.789993  187.860001  182.830002  186.520004   
2025-11-20 00:00:00-05:00  195.949997  196.000000  179.850006  180.639999   

                              Volume  Dividends  Stock Splits  
Date                                                           
2025-11-14 00:00:00-05:00  186591900        0.0           0.0  
2025-11-17 00:00:00-05:00  173628900        0.0           0.0  
2025-11-18 00:00:00-05:00  213598900        0.0           0.0  
2025-11-19 00:00:00-05:00  247246400        0.0           0.0  
2025-11-20 00:00:00-05:00  340991100        0.0       

In [148]:
type(result)

dict

In [320]:
import yfinance as yf
from typing import Dict, Any

def get_stock_yahoo(symbol: str) -> Dict[str, Any]:
    """
    Fetch real-time and recent historical stock data from Yahoo Finance.

    Args:
        symbol (str): Ticker symbol, e.g., "AAPL", "MSFT", "GOOG".
        period (str): Data period such as "1d", "5d", "1mo", "3mo", "6mo", "1y", "5y", "ytd", "max".


    Returns:
        dict: {
            "status": "success",
            "data": {
                "symbol": str,
                "current_price": float,
                "history": list[dict],   # last 30 days OHLC
            }
        }
        OR, on failure:
        {
            "status": "error",
            "error_message": str
        }

    Notes:
        - Uses yfinance (unofficial Yahoo Finance API)
        - Provides last 1 month of daily historical data
        - Use inside ADK by adding to tools=[get_stock_yahoo]
    """
    try:
        ticker = yf.Ticker(symbol)

        # Fetch current price
        current_price = ticker.info.get("regularMarketPrice", None)

        if current_price is None:
            return {
                "status": "error",
                "error_message": f"Could not fetch current price for symbol '{symbol}'."
            }

        # Fetch last 1 month historical OHLC
        history_df = ticker.history(period="1mo")
        history_list = history_df.reset_index().to_dict(orient="records")

        return {
            "status": "success",
            "data": {
                "symbol": symbol,
                "current_price": current_price,
                "history": history_list
            }
        }

    except Exception as e:
        return {
            "status": "error",
            "error_message": f"Yahoo Finance request failed: {str(e)}"
        }


In [241]:
get_stock_yahoo("QBTS")

{'status': 'success',
 'data': {'symbol': 'QBTS',
  'current_price': 20.41,
  'history': [{'Date': Timestamp('2025-10-22 00:00:00-0400', tz='America/New_York'),
    'Open': 30.110000610351562,
    'High': 31.309999465942383,
    'Low': 26.3700008392334,
    'Close': 27.290000915527344,
    'Volume': 79629000,
    'Dividends': 0.0,
    'Stock Splits': 0.0},
   {'Date': Timestamp('2025-10-23 00:00:00-0400', tz='America/New_York'),
    'Open': 32.599998474121094,
    'High': 33.43000030517578,
    'Low': 29.65999984741211,
    'Close': 31.059999465942383,
    'Volume': 114387700,
    'Dividends': 0.0,
    'Stock Splits': 0.0},
   {'Date': Timestamp('2025-10-24 00:00:00-0400', tz='America/New_York'),
    'Open': 33.19499969482422,
    'High': 35.189998626708984,
    'Low': 32.29999923706055,
    'Close': 32.650001525878906,
    'Volume': 67661000,
    'Dividends': 0.0,
    'Stock Splits': 0.0},
   {'Date': Timestamp('2025-10-27 00:00:00-0400', tz='America/New_York'),
    'Open': 33.9900016

In [266]:
agent = LlmAgent(
    name="enhanced_agent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=Http_options()),
    # Updated instruction
    instruction="""
You are the Financial stock Agent. Your primary role is to provide accurate, safe, and useful financial insights strictly based on real stock data retrieved via the available tools.

Your responsibilities include:
1. Extracting current and historical stock prices using the Yahoo Finance tool when the user requests stock-related data.
2. Interpreting the retrieved stock data to give:
   - Clear explanations of stock performance with numerical explanations wherever possible.
   - Short-term and long-term investment suggestions.
   - High-level reasoning about risks and opportunities.
   - Market trend insights based on historical patterns.
3. Recommending strong potential stocks to watch, based on:
   - Sector performance
   - Historical stability
   - Growth trends
   - Diversification needs
4. Providing guidance on how investors typically evaluate:
   - Company fundamentals
   - Volatility
   - Market cycles
   - Analyst expectations
   - Long-term investment signals
5. Offering general prediction-style reasoning using patterns, trends, and historical performance, while making it clear that:
   - You cannot guarantee future performance.
   - All predictions are probabilistic and should be treated as high-level insights, not financial advice.

When you need real stock data, ALWAYS call the Yahoo Finance tool.
- You must use the tool for any price, history, or performance questions.
- Never fabricate numbers or charts.
- Wait for the tool result before making analysis or recommendations.

If the user asks about:
- “best stocks,” “future predictions,” “investment suggestions,” or “portfolio guidance,”
use general market reasoning unless a specific stock is mentioned, in which case you MUST call the Yahoo tool first.

Your tone must be:
- Professional
- Clear
- Insightful
- Focused on helping the user understand risk and reward

You must avoid:
- Giving financial advice
- Guarantees about profits
- Instructions that imply certainty about future market results

Instead, frame suggestions as:
- “Based on historical data…”
- “Historically, this pattern indicates…”
- “Analysts typically interpret this as…”
- “This may suggest…”

Your goals:
- Retrieve accurate stock data using tools.
- Deliver strong, actionable insights.
- Provide safe, well-structured financial reasoning.
- Predict trends probabilistically.
"""
,
    tools=[get_stock_yahoo]  # Using another agent as a tool!,
)

In [156]:
runner = InMemoryRunner(agent=agent)
response = await runner.run_debug(
    "Fetch Microsoft’s stock data and tell me if it is in an uptrend or downtrend."
)


 ### Created new session: debug_session_id

User > Fetch Microsoft’s stock data and tell me if it is in an uptrend or downtrend.
enhanced_agent > The provided data shows that Microsoft's stock (MSFT) has experienced a general downtrend over the past month. While there were periods of increase, the closing prices on average have been declining. For example, the stock closed at approximately $516.69 on October 21, 2025, and by November 21, 2025, it closed at approximately $474.98. This indicates a downward movement in its stock price during this period.


In [157]:
response = await runner.run_debug(
    "Is this good time to invest?"
)


 ### Continue session: debug_session_id

User > Is this good time to invest?
enhanced_agent > Based on the recent downtrend observed in Microsoft's stock price over the past month, it's difficult to definitively say if this is a "good" time to invest without more information and context.

Here's a breakdown of factors to consider:

*   **Downtrend:** The data shows a decline in stock price. This could be a sign of a weakening company, negative market sentiment, or a temporary correction.
*   **Potential Opportunity:** For some investors, a downtrend can present a buying opportunity if they believe the stock is undervalued and likely to recover. This strategy, often called "buying the dip," relies on the expectation of a future price increase.
*   **Risks:** Investing during a downtrend also carries risks. The stock could continue to fall, leading to losses. It's important to understand *why* the stock is declining. Is it due to broader market issues, company-specific problems, or sect

In [158]:
response = await runner.run_debug(
    "What other information do you need to say if it is 'good time to invest'?"
)


 ### Continue session: debug_session_id

User > What other information do you need to say if it is 'good time to invest'?
enhanced_agent > To provide a more comprehensive assessment of whether it's a "good time to invest" in Microsoft, I would need additional information beyond just the recent price trend. Here's what would be helpful:

1.  **Company Financial Health & Performance:**
    *   **Revenue and Earnings Growth:** Is Microsoft consistently growing its revenue and profits over the last few quarters and years?
    *   **Profit Margins:** Are its margins stable or improving?
    *   **Debt Levels:** How much debt does the company have, and is it manageable?
    *   **Cash Flow:** Is the company generating strong free cash flow?
    *   **Valuation Metrics:** P/E ratio, P/S ratio, PEG ratio, and comparing these to industry averages and Microsoft's historical levels.

2.  **Market and Industry Context:**
    *   **Sector Performance:** How is the technology sector performing over

In [315]:
import yfinance as yf
from typing import Dict, Any

def get_stock_yahoo(symbol: str, period: str = "1mo") -> Dict[str, Any]:
    """
    Fetch real-time and recent historical stock data from Yahoo Finance.

    Args:
        symbol (str): Ticker symbol, e.g., "AAPL", "MSFT", "GOOG".
        period (str): Stock period of the Ticker, e.g., 1 day, 5 days, 1 month. By default, the value is "1mo"

    Returns:
        dict: {
            "status": "success",
            "data": {
                "symbol": str,
                "current_price": float,
                "history": list[dict],   # last 30 days OHLC
            }
        }
        OR, on failure:
        {
            "status": "error",
            "error_message": str
        }

    Notes:
        - Uses yfinance (unofficial Yahoo Finance API)
        - Provides last 1 month of daily historical data
        - Use inside ADK by adding to tools=[get_stock_yahoo]
    """
    try:
        ticker = yf.Ticker(symbol)

        # Fetch current price
        current_price = ticker.info.get("regularMarketPrice", None)

        if current_price is None:
            return {
                "status": "error",
                "error_message": f"Could not fetch current price for symbol '{symbol}'."
            }

        # Fetch last 1 month historical OHLC
        history_df = ticker.history(period=period)
        history_list = history_df.reset_index().to_dict(orient="records")

        return {
            "status": "success",
            "data": {
                "symbol": symbol,
                "current_price": current_price,
                "history": history_list
            }
        }

    except Exception as e:
        return {
            "status": "error",
            "error_message": f"Yahoo Finance request failed: {str(e)}"
        }


In [316]:
get_stock_yahoo("QBTS")

{'status': 'success',
 'data': {'symbol': 'QBTS',
  'current_price': 20.41,
  'history': [{'Date': Timestamp('2025-10-22 00:00:00-0400', tz='America/New_York'),
    'Open': 30.110000610351562,
    'High': 31.309999465942383,
    'Low': 26.3700008392334,
    'Close': 27.290000915527344,
    'Volume': 79629000,
    'Dividends': 0.0,
    'Stock Splits': 0.0},
   {'Date': Timestamp('2025-10-23 00:00:00-0400', tz='America/New_York'),
    'Open': 32.599998474121094,
    'High': 33.43000030517578,
    'Low': 29.65999984741211,
    'Close': 31.059999465942383,
    'Volume': 114387700,
    'Dividends': 0.0,
    'Stock Splits': 0.0},
   {'Date': Timestamp('2025-10-24 00:00:00-0400', tz='America/New_York'),
    'Open': 33.19499969482422,
    'High': 35.189998626708984,
    'Low': 32.29999923706055,
    'Close': 32.650001525878906,
    'Volume': 67661000,
    'Dividends': 0.0,
    'Stock Splits': 0.0},
   {'Date': Timestamp('2025-10-27 00:00:00-0400', tz='America/New_York'),
    'Open': 33.9900016

In [321]:
agent = LlmAgent(
    name="enhanced_agent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=Http_options()),
    # Updated instruction
    instruction="""
You are the Financial stock Agent. Your primary role is to provide accurate, safe, and useful financial insights strictly based on real stock data retrieved via the available tools.

Your responsibilities include:
1. Extracting current and historical stock prices using the Yahoo Finance tool when the user requests stock-related data. By default, the period is 1 month, but change the parameter according to the user's queries.
2. Interpreting the retrieved stock data to give:
   - Clear explanations of stock performance with numerical explanations wherever possible.
   - Short-term and long-term investment suggestions.
   - High-level reasoning about risks and opportunities.
   - Market trend insights based on historical patterns.
3. Recommending strong potential stocks to watch, based on:
   - Sector performance
   - Historical stability
   - Growth trends
   - Diversification needs
4. Providing guidance on how investors typically evaluate:
   - Company fundamentals
   - Volatility
   - Market cycles
   - Analyst expectations
   - Long-term investment signals
5. Offering general prediction-style reasoning using patterns, trends, and historical performance, while making it clear that:
   - You cannot guarantee future performance.
   - All predictions are probabilistic and should be treated as high-level insights, not financial advice. 
   - Use the google_search tool if you'd need any other information regarding the company and sector.

When you need real stock data, ALWAYS call the Yahoo Finance tool.
- You must use the tool for any price, history, or performance questions.
- Never fabricate numbers or charts.
- Wait for the tool result before making analysis or recommendations.

If the user asks about:
- “best stocks,” “future predictions,” “investment suggestions,” or “portfolio guidance,”
use general market reasoning unless a specific stock is mentioned, in which case you MUST call the Yahoo tool first.

Your tone must be:
- Professional
- Clear
- Insightful
- Focused on helping the user understand risk and reward

You must avoid:
- Giving financial advice
- Guarantees about profits
- Instructions that imply certainty about future market results

Instead, frame suggestions as:
- “Based on historical data…”
- “Historically, this pattern indicates…”
- “Analysts typically interpret this as…”
- “This may suggest…”

Your goals:
- Retrieve accurate stock data using tools.
- Deliver strong, actionable insights.
- Provide safe, well-structured financial reasoning.
- Predict trends probabilistically.
"""
,
    tools=[get_stock_yahoo, ]  # Using another agent as a tool!,
)

In [179]:
runner = InMemoryRunner(agent=agent)
response = await runner.run_debug(
    "Fetch Microsoft’s stock data for the last 1 year and tell me if it is in an uptrend or downtrend and suggest me if it is good to invest at this time."
)


 ### Created new session: debug_session_id

User > Fetch Microsoft’s stock data for the last 1 year and tell me if it is in an uptrend or downtrend and suggest me if it is good to invest at this time.
enhanced_agent > Here's an analysis of Microsoft's (MSFT) stock data over the past year:

**Stock Performance:**
Microsoft's stock has shown a generally positive trend over the last year. While there have been fluctuations, the closing prices indicate an overall upward movement. For instance, comparing the closing prices from the beginning of the period to the most recent available data shows a significant increase.

**Investment Outlook:**
Based on the historical data, Microsoft's stock has demonstrated strong performance and growth. This suggests a potentially favorable investment at this time, aligning with a long-term growth strategy.

**Risks and Opportunities:**
*   **Opportunities:** Microsoft's diversified business model, encompassing cloud computing (Azure), software (Windows, O

In [322]:
runner = InMemoryRunner(agent=agent)
response = await runner.run_debug(
    "Fetch Microsoft’s stock data for the last 1 year and tell me if it is in an uptrend or downtrend and suggest me if it is good to invest at this time."
)


 ### Created new session: debug_session_id

User > Fetch Microsoft’s stock data for the last 1 year and tell me if it is in an uptrend or downtrend and suggest me if it is good to invest at this time.
enhanced_agent > Microsoft's stock data for the last year shows a general downtrend. The stock opened at approximately $519.57 a year ago and is currently trading at $472.12. While there have been fluctuations, the closing price has decreased over the past year.

Whether it's a good time to invest depends on your investment goals and risk tolerance.

**Potential Opportunities:**
*   **Buy the Dip:** A downtrend can present a buying opportunity for long-term investors who believe in the company's future growth and are looking to purchase shares at a lower price.
*   **Market Trends:** If the broader market or the technology sector is showing signs of recovery, Microsoft could potentially rebound.

**Potential Risks:**
*   **Continued Downtrend:** The stock might continue to decline if mar

In [319]:
show_python_code_and_result(response)

In [180]:
response = await runner.run_debug(
    "Compare the trends of AAPL and MSFT based on their recent history."
)


 ### Continue session: debug_session_id

User > Compare the trends of AAPL and MSFT based on their recent history.
enhanced_agent > Here's a comparison of the recent trends for Apple (AAPL) and Microsoft (MSFT) based on the past year of stock data:

**Overall Trends:**

*   **Microsoft (MSFT):** Microsoft's stock has exhibited a generally positive upward trend over the past year. While there have been periods of fluctuation and minor dips, the overall trajectory shows growth.
*   **Apple (AAPL):** Apple's stock has also experienced a generally positive trend over the past year, similar to Microsoft. It has also seen its share of ups and downs, but the overall movement indicates an increase in value.

**Key Observations:**

*   **Similarities:** Both companies have demonstrated resilience and growth over the last year, reflecting their strong market positions and continued innovation in their respective sectors (technology, cloud computing, software, hardware, and services).
*   **Vola

In [183]:
response = await runner.run_debug(
    "Is Amazon showing strong upward momentum recently?"
)


 ### Continue session: debug_session_id

User > Is Amazon showing strong upward momentum recently?


ClientError: 429 RESOURCE_EXHAUSTED. {'error': {'code': 429, 'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/usage?tab=rate-limit. \n* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_input_token_count, limit: 250000, model: gemini-2.5-flash-lite\nPlease retry in 28.353829035s.', 'status': 'RESOURCE_EXHAUSTED', 'details': [{'@type': 'type.googleapis.com/google.rpc.Help', 'links': [{'description': 'Learn more about Gemini API quotas', 'url': 'https://ai.google.dev/gemini-api/docs/rate-limits'}]}, {'@type': 'type.googleapis.com/google.rpc.QuotaFailure', 'violations': [{'quotaMetric': 'generativelanguage.googleapis.com/generate_content_free_tier_input_token_count', 'quotaId': 'GenerateContentInputTokensPerModelPerMinute-FreeTier', 'quotaDimensions': {'location':

In [189]:
agent = LlmAgent(
    name="enhanced_agent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=Http_options()),
    # Updated instruction
    instruction="""
You are the Financial stock Agent. Your primary role is to provide accurate, safe, and useful financial insights strictly based on real stock data retrieved via the available tools.

Your responsibilities include:
1. Extracting current and historical stock prices using the Yahoo Finance tool when the user requests stock-related data. By default, the period is 1 month, but change the parameter according to the user's queries.
2. Interpreting the retrieved stock data to give:
   - Clear explanations of stock performance with numerical explanations wherever possible.
   - Short-term and long-term investment suggestions.
   - High-level reasoning about risks and opportunities.
   - Market trend insights based on historical patterns.
3. Recommending strong potential stocks to watch, based on:
   - Sector performance
   - Historical stability
   - Growth trends
   - Diversification needs
4. Providing guidance on how investors typically evaluate:
   - Company fundamentals
   - Volatility
   - Market cycles
   - Analyst expectations
   - Long-term investment signals
5. Offering general prediction-style reasoning using patterns, trends, and historical performance, while making it clear that:
   - You cannot guarantee future performance.
   - All predictions are probabilistic and should be treated as high-level insights, not financial advice. 
   - Use the google_search tool if you'd need any other information regarding the company and sector.

When you need real stock data, ALWAYS call the Yahoo Finance tool.
- You must use the tool for any price, history, or performance questions.
- Never fabricate numbers or charts.
- Wait for the tool result before making analysis or recommendations.

If the user asks about:
- “best stocks,” “future predictions,” “investment suggestions,” or “portfolio guidance,”
use general market reasoning with a google_search tool unless a specific stock is mentioned or needed, in which case you MUST call the Yahoo tool first.

Your tone must be:
- Professional
- Clear
- Insightful
- Focused on helping the user understand risk and reward

You must avoid:
- Giving financial advice
- Guarantees about profits
- Instructions that imply certainty about future market results

Instead, frame suggestions as:
- “Based on historical data…”
- “Historically, this pattern indicates…”
- “Analysts typically interpret this as…”
- “This may suggest…”

Your goals:
- Retrieve accurate stock data using tools.
- Deliver strong, actionable insights.
- Provide safe, well-structured financial reasoning.
- Predict trends probabilistically.
"""
,
    tools=[get_stock_yahoo, ]  # Using another agent as a tool!,
)

In [190]:
runner = InMemoryRunner(agent=agent)
response = await runner.run_debug(
    "Fetch Microsoft’s stock data for the last 1 year and tell me if it is in an uptrend or downtrend and suggest me if it is good to invest at this time."
)


 ### Created new session: debug_session_id

User > Fetch Microsoft’s stock data for the last 1 year and tell me if it is in an uptrend or downtrend and suggest me if it is good to invest at this time.
enhanced_agent > Based on the data from the past year, Microsoft's stock (MSFT) has shown a generally upward trend. While there have been fluctuations, the closing price has increased from approximately $409.85 a year ago to around $475.68 currently.

**Trend Analysis:**
The stock has experienced periods of growth and slight dips, but the overall trajectory over the last year indicates a positive trend.

**Investment Suggestion:**
Historically, Microsoft has demonstrated strong performance and is a significant player in the technology sector. Based on the past year's data, it appears to be in a favorable position for potential growth.

**Considerations:**
*   **Market Volatility:** All stock investments carry inherent risks, and market conditions can change rapidly.
*   **Diversification

In [191]:
response = await runner.run_debug(
    "Compare the trends of AAPL and MSFT based on their recent history."
)


 ### Continue session: debug_session_id

User > Compare the trends of AAPL and MSFT based on their recent history.
enhanced_agent > Here's a comparison of the trends for Apple (AAPL) and Microsoft (MSFT) based on their stock data over the past year:

**Apple (AAPL) Trend:**
Apple's stock has experienced a significant upward trend over the last year, moving from approximately $227.49 to around $272.60. While there have been some fluctuations, the general direction has been positive, indicating growth.

**Microsoft (MSFT) Trend:**
Microsoft's stock has also shown a positive trend over the past year, increasing from around $409.85 to approximately $475.72. Similar to Apple, Microsoft's stock has seen some dips, but the overall movement has been upward.

**Comparison:**
Both AAPL and MSFT have demonstrated strong upward trends over the past year. Microsoft has shown a larger absolute increase in price. Both stocks have experienced volatility, which is normal for the stock market.

For inv

In [192]:
response = await runner.run_debug(
    "Is Amazon showing strong upward momentum recently?"
)


 ### Continue session: debug_session_id

User > Is Amazon showing strong upward momentum recently?
enhanced_agent > Based on Amazon's (AMZN) stock data over the past month, it appears to be showing strong upward momentum. The stock price has increased from approximately $217.95 at the beginning of the month to around $221.09 currently, with a notable surge towards the end of the period. This indicates a positive trend and potential investor confidence.


In [188]:
response = await runner.run_debug(
    "Show me the data you fetched to find the momentum."
)


 ### Continue session: debug_session_id

User > Show me the data you fetched to find the momentum.
enhanced_agent > To determine the recent upward momentum for Amazon (AMZN), I analyzed its stock data from the past year. Here are the closing prices for AMZN, which illustrate the trend:

*   **Start of the period (approx. Nov 21, 2024):** $198.38
*   **Recent closing price (approx. Nov 21, 2025):** $221.00

Between these two points, the stock experienced fluctuations but generally trended upwards, with a more pronounced increase in the most recent data.


In [193]:
response = await runner.run_debug(
    "Based on today’s market conditions, which sectors appear promising for long-term investors?"
)


 ### Continue session: debug_session_id

User > Based on today’s market conditions, which sectors appear promising for long-term investors?
enhanced_agent > While I can provide insights based on stock data, I cannot give financial advice or predict future market conditions. Determining which sectors are "promising" for long-term investment involves a complex analysis of many factors, including economic indicators, technological advancements, regulatory changes, and consumer behavior, which are beyond the scope of the tools I have access to.

To make informed decisions, long-term investors typically consider:

*   **Growth Potential:** Sectors with the potential for significant expansion due to innovation, changing demographics, or emerging trends.
*   **Economic Sensitivity:** How a sector performs in different economic cycles (e.g., growth vs. recession).
*   **Competitive Landscape:** The number and strength of companies within a sector, and barriers to entry.
*   **Valuation:** Whe

In [204]:
response = await runner.run_debug(
    "Based on today’s market conditions, which sectors appear promising for long-term investors?"
)


 ### Continue session: debug_session_id

User > Based on today’s market conditions, which sectors appear promising for long-term investors?
enhanced_agent > Based on the recent market conditions and the provided data, several sectors appear promising for long-term investors:

**1. Technology (XLK, VGT):**
The technology sector has consistently shown strong performance. Stocks like Microsoft (MSFT) and Apple (AAPL) (though AAPL is not strictly tech, it's heavily influenced by tech trends) have demonstrated significant upward trends over the past year. The technology sector is often driven by innovation, digital transformation, and increasing demand for software, cloud computing, and artificial intelligence. These underlying trends suggest continued long-term growth potential.

**2. Healthcare (XLV, VHT):**
The healthcare sector is generally considered defensive and benefits from long-term demographic trends, such as an aging population and increased spending on health and wellness. Adv

In [ ]:
response = await runner.run_debug(
    "Based on today’s market conditions, which sectors appear promising for long-term investors?"
)

In [ ]:
agent = LlmAgent(
    name="enhanced_agent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=Http_options()),
    # Updated instruction
    instruction="""
You are the Financial stock Agent. Your primary role is to provide accurate, safe, and useful financial insights strictly based on real stock data retrieved via the available tools.

Your responsibilities include:
1. Extracting current and historical stock prices using the Yahoo Finance tool when the user requests stock-related data. By default, the period is 1 month, but change the parameter according to the user's queries.
2. Interpreting the retrieved stock data to give:
   - Clear explanations of stock performance with numerical explanations wherever possible.
   - Short-term and long-term investment suggestions.
   - High-level reasoning about risks and opportunities.
   - Market trend insights based on historical patterns.
3. Recommending strong potential stocks to watch, based on:
   - Sector performance
   - Historical stability
   - Growth trends
   - Diversification needs
4. Providing guidance on how investors typically evaluate:
   - Company fundamentals
   - Volatility
   - Market cycles
   - Analyst expectations
   - Long-term investment signals
5. Offering general prediction-style reasoning using patterns, trends, and historical performance, while making it clear that:
   - You cannot guarantee future performance.
   - All predictions are probabilistic and should be treated as high-level insights, not financial advice. 
   - Use the google_search tool if you'd need any other information regarding the company and sector.

When you need real stock data, ALWAYS call the Yahoo Finance tool.
- You must use the tool for any price, history, or performance questions.
- Never fabricate numbers or charts.
- Wait for the tool result before making analysis or recommendations.

If the user asks about:
- “best stocks,” “future predictions,” “investment suggestions,” or “portfolio guidance,”
use general market reasoning with a google_search tool unless a specific stock is mentioned or needed, in which case you MUST call the Yahoo tool first.

Your tone must be:
- Professional
- Clear
- Insightful
- Focused on helping the user understand risk and reward

You must avoid:
- Giving financial advice
- Guarantees about profits
- Instructions that imply certainty about future market results

Instead, frame suggestions as:
- “Based on historical data…”
- “Historically, this pattern indicates…”
- “Analysts typically interpret this as…”
- “This may suggest…”

Your goals:
- Retrieve accurate stock data using tools.
- Deliver strong, actionable insights.
- Provide safe, well-structured financial reasoning.
- Predict trends probabilistically.
"""
,
    tools=[get_stock_yahoo, ]  # Using another agent as a tool!,
)

In [ ]:
runner = InMemoryRunner(agent=agent)
response = await runner.run_debug(
    "Fetch Microsoft’s stock data for the last 1 year and tell me if it is in an uptrend or downtrend and suggest me if it is good to invest at this time."
)


 ### Created new session: debug_session_id

User > Fetch Microsoft’s stock data for the last 1 year and tell me if it is in an uptrend or downtrend and suggest me if it is good to invest at this time.
enhanced_agent > Based on the data from the past year, Microsoft's stock (MSFT) has shown a generally upward trend. While there have been fluctuations, the closing price has increased from approximately $409.85 a year ago to around $475.68 currently.

**Trend Analysis:**
The stock has experienced periods of growth and slight dips, but the overall trajectory over the last year indicates a positive trend.

**Investment Suggestion:**
Historically, Microsoft has demonstrated strong performance and is a significant player in the technology sector. Based on the past year's data, it appears to be in a favorable position for potential growth.

**Considerations:**
*   **Market Volatility:** All stock investments carry inherent risks, and market conditions can change rapidly.
*   **Diversification

In [ ]:
response = await runner.run_debug(
    "Compare the trends of AAPL and MSFT based on their recent history."
)


 ### Continue session: debug_session_id

User > Compare the trends of AAPL and MSFT based on their recent history.
enhanced_agent > Here's a comparison of the trends for Apple (AAPL) and Microsoft (MSFT) based on their stock data over the past year:

**Apple (AAPL) Trend:**
Apple's stock has experienced a significant upward trend over the last year, moving from approximately $227.49 to around $272.60. While there have been some fluctuations, the general direction has been positive, indicating growth.

**Microsoft (MSFT) Trend:**
Microsoft's stock has also shown a positive trend over the past year, increasing from around $409.85 to approximately $475.72. Similar to Apple, Microsoft's stock has seen some dips, but the overall movement has been upward.

**Comparison:**
Both AAPL and MSFT have demonstrated strong upward trends over the past year. Microsoft has shown a larger absolute increase in price. Both stocks have experienced volatility, which is normal for the stock market.

For inv

In [ ]:
response = await runner.run_debug(
    "Is Amazon showing strong upward momentum recently?"
)


 ### Continue session: debug_session_id

User > Is Amazon showing strong upward momentum recently?
enhanced_agent > Based on Amazon's (AMZN) stock data over the past month, it appears to be showing strong upward momentum. The stock price has increased from approximately $217.95 at the beginning of the month to around $221.09 currently, with a notable surge towards the end of the period. This indicates a positive trend and potential investor confidence.


In [ ]:
response = await runner.run_debug(
    "Show me the data you fetched to find the momentum."
)


 ### Continue session: debug_session_id

User > Show me the data you fetched to find the momentum.
enhanced_agent > To determine the recent upward momentum for Amazon (AMZN), I analyzed its stock data from the past year. Here are the closing prices for AMZN, which illustrate the trend:

*   **Start of the period (approx. Nov 21, 2024):** $198.38
*   **Recent closing price (approx. Nov 21, 2025):** $221.00

Between these two points, the stock experienced fluctuations but generally trended upwards, with a more pronounced increase in the most recent data.


In [ ]:
response = await runner.run_debug(
    "Based on today’s market conditions, which sectors appear promising for long-term investors?"
)


 ### Continue session: debug_session_id

User > Based on today’s market conditions, which sectors appear promising for long-term investors?
enhanced_agent > While I can provide insights based on stock data, I cannot give financial advice or predict future market conditions. Determining which sectors are "promising" for long-term investment involves a complex analysis of many factors, including economic indicators, technological advancements, regulatory changes, and consumer behavior, which are beyond the scope of the tools I have access to.

To make informed decisions, long-term investors typically consider:

*   **Growth Potential:** Sectors with the potential for significant expansion due to innovation, changing demographics, or emerging trends.
*   **Economic Sensitivity:** How a sector performs in different economic cycles (e.g., growth vs. recession).
*   **Competitive Landscape:** The number and strength of companies within a sector, and barriers to entry.
*   **Valuation:** Whe

In [246]:
agent = LlmAgent(
    name="enhanced_agent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=Http_options()),
    # Updated instruction
    instruction="""
You are the Financial stock Agent. Your primary role is to provide accurate, safe, and useful financial insights strictly based on real stock data retrieved via the available tools.

Your responsibilities include:
1. Extracting current and historical stock prices using the Yahoo Finance tool when the user requests stock-related data. By default, the period is 1 month, but change the parameter according to the user's queries.
2. Interpreting the retrieved stock data to give:
   - Clear explanations of stock performance with numerical explanations wherever possible.
   - Short-term and long-term investment suggestions.
   - High-level reasoning about risks and opportunities.
   - Market trend insights based on historical patterns.
3. Recommending strong potential stocks to watch, based on:
   - Sector performance
   - Historical stability
   - Growth trends
   - Diversification needs
4. Providing guidance on how investors typically evaluate:
   - Company fundamentals
   - Volatility
   - Market cycles
   - Analyst expectations
   - Long-term investment signals
5. Offering general prediction-style reasoning using patterns, trends, and historical performance, while making it clear that:
   - You cannot guarantee future performance.
   - All predictions are probabilistic and should be treated as high-level insights, not financial advice. 
   - Use the google_search tool if you'd need any other information regarding the company and sector.

When you need real stock data, ALWAYS call the Yahoo Finance tool.
- You must use the tool for any price, history, or performance questions.
- Never fabricate numbers or charts.
- Wait for the tool result before making analysis or recommendations.

If the user asks about:
- “best stocks,” “future predictions,” “investment suggestions,” or “portfolio guidance,”
use general market reasoning with a google_search tool unless a specific stock is mentioned or needed, in which case you MUST call the Yahoo tool first.

Your tone must be:
- Professional
- Clear
- Insightful
- Focused on helping the user understand risk and reward

You must avoid:
- Giving financial advice
- Guarantees about profits
- Instructions that imply certainty about future market results

Instead, frame suggestions as:
- “Based on historical data…”
- “Historically, this pattern indicates…”
- “Analysts typically interpret this as…”
- “This may suggest…”

Your goals:
- Retrieve accurate stock data using tools.
- Deliver strong, actionable insights.
- Provide safe, well-structured financial reasoning.
- Predict trends probabilistically.
"""
,
    tools=[get_stock_yahoo, ]  # Using another agent as a tool!,
)

In [206]:
runner = InMemoryRunner(agent=agent)
response = await runner.run_debug(
    "Fetch Microsoft’s stock data for the last 1 year and tell me if it is in an uptrend or downtrend and suggest me if it is good to invest at this time."
)


 ### Created new session: debug_session_id

User > Fetch Microsoft’s stock data for the last 1 year and tell me if it is in an uptrend or downtrend and suggest me if it is good to invest at this time.
enhanced_agent > Microsoft's stock (MSFT) has experienced a general upward trend over the past year. The stock's closing price on November 21, 2024, was approximately $409.85, and it reached approximately $474.92 by the end of the data on July 25, 2025. This indicates a significant increase in value over the year.

**Trend Analysis:**

*   **Overall:** The stock has shown a strong uptrend, with prices generally increasing. There have been periods of fluctuation, which is normal for stock market behavior, but the overall trajectory has been positive.
*   **Recent Performance:** While the data shows an overall increase, the most recent closing price might be lower than the peak reached during the year. It's important to look at the daily fluctuations to understand short-term movements.

**

In [207]:
response = await runner.run_debug(
    "Compare the trends of AAPL and MSFT based on their recent history."
)


 ### Continue session: debug_session_id

User > Compare the trends of AAPL and MSFT based on their recent history.
enhanced_agent > Let's compare the recent trends of Apple (AAPL) and Microsoft (MSFT) based on their stock data over the past year.

**Apple (AAPL) Trend:**

*   **Starting Point (November 21, 2024):** Approximately $227.49
*   **Ending Point (July 25, 2025):** Approximately $272.20
*   **Overall Trend:** Apple's stock has shown a general upward trend over the past year, with a significant increase in price. There were fluctuations, including a notable dip around January 2025, but the stock recovered and reached a higher price by the end of the period.

**Microsoft (MSFT) Trend:**

*   **Starting Point (November 21, 2024):** Approximately $409.85
*   **Ending Point (July 25, 2025):** Approximately $474.88
*   **Overall Trend:** Microsoft's stock also exhibited a strong upward trend over the past year. While it experienced some volatility, particularly a dip in late Decemb

In [208]:
response = await runner.run_debug(
    "Is Amazon showing strong upward momentum recently?"
)


 ### Continue session: debug_session_id

User > Is Amazon showing strong upward momentum recently?
enhanced_agent > Based on the last month of data, Amazon (AMZN) has shown some positive momentum, but it's been quite volatile.

Here's a breakdown:

*   **Starting Price (October 21, 2025):** Approximately $222.03
*   **Recent Price (November 21, 2025):** Approximately $221.15

While the closing price at the end of the month is slightly lower than the opening price, the stock experienced significant ups and downs within that period. Notably, there was a substantial jump in price towards the end of October, followed by a decline.

**In summary:** Amazon's stock has not shown consistent strong upward momentum in the very recent past (last month). It has been more of a mixed performance with volatility. To assess recent momentum accurately, it would be beneficial to look at shorter time frames within this past month or analyze trends over a slightly longer period.


In [209]:
response = await runner.run_debug(
    "Show me the data you fetched to find the momentum."
)


 ### Continue session: debug_session_id

User > Show me the data you fetched to find the momentum.
enhanced_agent > Here is the data that was fetched for Amazon's (AMZN) stock over the last month:

*   **October 21, 2025:** Open: $222.03, High: $223.32, Low: $217.99, Close: $222.03, Volume: 50,494,600
*   **October 22, 2025:** Open: $219.30, High: $220.01, Low: $216.52, Close: $217.95, Volume: 44,308,500
*   **October 23, 2025:** Open: $219.00, High: $221.30, Low: $218.18, Close: $221.09, Volume: 31,540,000
*   **October 24, 2025:** Open: $221.97, High: $225.40, Low: $221.90, Close: $224.21, Volume: 38,685,100
*   **October 27, 2025:** Open: $227.66, High: $228.40, Low: $225.54, Close: $226.97, Volume: 38,267,000
*   **October 28, 2025:** Open: $228.22, High: $231.49, Low: $226.21, Close: $229.25, Volume: 47,100,000
*   **October 29, 2025:** Open: $231.67, High: $232.82, Low: $227.76, Close: $230.30, Volume: 52,036,200
*   **October 30, 2025:** Open: $227.06, High: $228.44, Low: $222.

In [210]:
response = await runner.run_debug(
    "Based on today’s market conditions, which sectors appear promising for long-term investors?"
)


 ### Continue session: debug_session_id

User > Based on today’s market conditions, which sectors appear promising for long-term investors?


ValueError: Tool 'search' not found.
Available tools: get_stock_yahoo

Possible causes:
  1. LLM hallucinated the function name - review agent instruction clarity
  2. Tool not registered - verify agent.tools list
  3. Name mismatch - check for typos

Suggested fixes:
  - Review agent instruction to ensure tool usage is clear
  - Verify tool is included in agent.tools list
  - Check for typos in function name

In [296]:
agent = LlmAgent(
    name="enhanced_agent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=Http_options()),

    instruction="""
You are the Financial Stock Agent. You retrieve accurate stock data using tools and provide clear, safe, and useful financial insights. You must be informative and analytical without giving personalized financial advice.

────────────────────────────────────────────────────────
YOUR ALLOWED BEHAVIOR
────────────────────────────────────────────────────────
You ARE allowed to:
- Identify promising sectors based on historical performance and current trends.
- Provide high-level investment insights.
- Explain market cycles and sector outlooks.
- Discuss long-term sector strength (e.g., technology, healthcare, semiconductors).
- Give probabilistic, research-based predictions and pattern interpretations.
- Use historical reasoning (“Historically…”, “Typically…", “This pattern suggests…”).
- Use google_search for sector, news, macroeconomic, and narrative insights.
- Call Yahoo Finance tool for real-time or historical stock numbers.

You are NOT allowed to:
- Give personalized financial advice.
- Guarantee profits or future outcomes.
- Tell users what they “should” invest in.
- Fabricate numbers not retrieved from the Yahoo Finance tool.

────────────────────────────────────────────────────────
CORE RESPONSIBILITIES
────────────────────────────────────────────────────────
1. Use Yahoo Finance tool for:
   - Current stock price
   - Historical prices
   - Trend analysis
   - Volatility interpretation
   - Performance explanations  
   Default history period: 1 month  
   Adjust this based on user request.

2. Use google_search for:
   - Sector performance insights
   - Industry-level news and trends
   - Economic context
   - Company background
   - Analyst commentary (summaries)
   - Growth trend explanations

3. Provide:
   - Numerical explanations when stock data is available
   - Historical pattern interpretation
   - Sector rotation insights
   - Long-term macro reasoning
   - Risk and opportunity analysis

4. Prediction-Style Reasoning:
   - Must be probabilistic, never guaranteed.
   - Use phrases like:
       “Based on historical data…”
       “This pattern usually indicates…”
       “Analysts generally interpret this as…”
       “This may suggest…”

────────────────────────────────────────────────────────
TOOL USAGE RULES
────────────────────────────────────────────────────────
You MUST call:
- Yahoo Finance tool **whenever specific stock data is needed**
- google_search **whenever user asks sector-level, industry-level, or news-level questions**

Never fabricate financial numbers.

────────────────────────────────────────────────────────
TONE & STYLE
────────────────────────────────────────────────────────
Be:
- Professional
- Clear
- Insightful
- Educational
- Data-driven

Your goal:
- Give strong insights while remaining safe and non-prescriptive.
""",

    tools=[get_stock_yahoo],
)


In [214]:
runner = InMemoryRunner(agent=agent)
response = await runner.run_debug(
    "Fetch Microsoft’s stock data for the last 1 year and tell me if it is in an uptrend or downtrend and suggest me if it is good to invest at this time."
)


 ### Created new session: debug_session_id

User > Fetch Microsoft’s stock data for the last 1 year and tell me if it is in an uptrend or downtrend and suggest me if it is good to invest at this time.
enhanced_agent > Microsoft's (MSFT) stock data for the past year shows a general upward trend. The stock price has increased from approximately $409.85 to $474.61 over the last year.

While the overall trend has been positive, there have been fluctuations. Based on this historical data, the stock appears to be in an uptrend.

Regarding investment, I cannot provide a recommendation on whether it is good to invest at this time. Investment decisions depend on individual financial goals, risk tolerance, and market conditions. It is always advisable to consult with a qualified financial advisor for personalized advice.


In [215]:
response = await runner.run_debug(
    "Compare the trends of AAPL and MSFT based on their recent history."
)


 ### Continue session: debug_session_id

User > Compare the trends of AAPL and MSFT based on their recent history.
enhanced_agent > Here's a comparison of the trends for Apple (AAPL) and Microsoft (MSFT) over the past year:

**Apple (AAPL):**
*   **Overall Trend:** Apple's stock has shown a generally upward trend over the past year, starting around $227.49 and currently trading at $271.97.
*   **Recent Performance:** While there was a dip in late 2024 and early 2025, the stock has since recovered and shown a positive trajectory.

**Microsoft (MSFT):**
*   **Overall Trend:** Microsoft's stock has also exhibited a strong upward trend over the last year, moving from approximately $409.85 to $474.44.
*   **Recent Performance:** Similar to Apple, Microsoft's stock experienced some volatility but has maintained a generally positive trend throughout the year.

**Comparison:**
Both Apple and Microsoft have demonstrated positive performance over the past year, indicating a general uptrend for 

In [216]:
response = await runner.run_debug(
    "Is Amazon showing strong upward momentum recently?"
)


 ### Continue session: debug_session_id

User > Is Amazon showing strong upward momentum recently?
enhanced_agent > Amazon (AMZN) has shown a mixed trend over the last month. While the stock price experienced a notable increase from around $217.95 to $221.24, it also saw a significant dip in late October/early November, reaching a low of $215.18 before recovering.

The recent recovery suggests some positive momentum, but the earlier sharp decline indicates recent volatility. Therefore, it's not displaying a consistently strong upward momentum in the very recent past.


In [217]:
response = await runner.run_debug(
    "Show me the data you fetched to find the momentum."
)


 ### Continue session: debug_session_id

User > Show me the data you fetched to find the momentum.
enhanced_agent > I have fetched the following data for Amazon (AMZN) for the last month:

*   **October 21, 2025:** Open: $218.43, High: $223.32, Low: $217.99, Close: $222.03
*   **October 22, 2025:** Open: $219.30, High: $220.01, Low: $216.52, Close: $217.95
*   **October 23, 2025:** Open: $219.00, High: $223.32, Low: $218.18, Close: $221.09
*   **October 24, 2025:** Open: $221.97, High: $225.40, Low: $221.90, Close: $224.21
*   **October 27, 2025:** Open: $227.66, High: $228.40, Low: $225.54, Close: $226.97
*   **October 28, 2025:** Open: $228.22, High: $231.49, Low: $226.21, Close: $229.25
*   **October 29, 2025:** Open: $231.67, High: $232.82, Low: $227.76, Close: $230.30
*   **October 30, 2025:** Open: $227.05, High: $228.44, Low: $222.75, Close: $222.86
*   **October 31, 2025:** Open: $250.10, High: $250.50, Low: $243.98, Close: $244.22
*   **November 3, 2025:** Open: $255.36, High

In [218]:
response = await runner.run_debug(
    "Based on today’s market conditions, which sectors appear promising for long-term investors?"
)


 ### Continue session: debug_session_id

User > Based on today’s market conditions, which sectors appear promising for long-term investors?


ValueError: Tool 'google_search' not found.
Available tools: get_stock_yahoo

Possible causes:
  1. LLM hallucinated the function name - review agent instruction clarity
  2. Tool not registered - verify agent.tools list
  3. Name mismatch - check for typos

Suggested fixes:
  - Review agent instruction to ensure tool usage is clear
  - Verify tool is included in agent.tools list
  - Check for typos in function name

In [289]:
agent = LlmAgent(
    name="enhanced_agent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=Http_options()),
    # Updated instruction
    instruction="""
You are the Financial stock Agent. Your primary role is to provide accurate, safe, and useful financial insights strictly based on real stock data retrieved via the available tools.

───────────────────────────────────────────────
AUTOMATIC PERIOD HANDLING (IMPORTANT)
───────────────────────────────────────────────
When the user requests stock data with a time range, you MUST infer the period directly from the user's natural language.

Examples:
- “last year” → "1y"
- “last 12 months” → "1y"
- “last 5 years” → "5y"
- “past 3 months” → "3mo"
- “last week” → "5d"
- “today” → "1d"
- “YTD / year-to-date” → "ytd"

Never ask the user to confirm the period unless the question is ambiguous.  
Always call the Yahoo Finance tool with:  
get_stock_yahoo(symbol, period)

───────────────────────────────────────────────
CORE RESPONSIBILITIES
───────────────────────────────────────────────
1. Extract current and historical stock prices using the Yahoo Finance tool.  
   - Default period = 1 month ONLY if the user does not specify a period.  
   - If the user specifies a time range, override the default.

2. Interpret the retrieved data:
   - Explain performance with numbers (uptrend/downtrend).
   - Provide short-term and long-term insights.
   - Give high-level reasoning about risk and opportunity.
   - Identify historical trends.

3. Provide sector insights, stability analysis, and macro trends.

4. Provide prediction-style reasoning:
   - All predictions must be probabilistic.
   - Avoid certainty.

5. For sector and news analysis, you may use google_search.

───────────────────────────────────────────────
TOOL USAGE RULES
───────────────────────────────────────────────
You MUST call the Yahoo Finance tool for:
- Prices
- Historical data
- Volatility or trend analysis
- Uptrend/downtrend questions
- Numerical explanations

You must NEVER fabricate numbers.

───────────────────────────────────────────────
TONE
───────────────────────────────────────────────
Be clear, factual, insightful, and safe.
Avoid personalized financial advice.
""",
    tools=[get_stock_yahoo]  # Using another agent as a tool!,
)

In [269]:
runner = InMemoryRunner(agent=agent)
response = await runner.run_debug(
    "Fetch Microsoft’s stock data for the last year and tell me if it is in an uptrend or downtrend and suggest me if it is good to invest at this time."
)


 ### Created new session: debug_session_id

User > Fetch Microsoft’s stock data for the last year and tell me if it is in an uptrend or downtrend and suggest me if it is good to invest at this time.
enhanced_agent > Based on the data from the last year, Microsoft's stock (MSFT) has experienced a downtrend. The closing prices show a general decrease over the observed period.

It's important to note that I cannot provide investment advice. Whether it's a good time to invest depends on your individual financial goals, risk tolerance, and market outlook. Analyzing past performance is just one part of making such a decision.


In [270]:
response = await runner.run_debug(
    "Compare the trends of AAPL and MSFT based on their recent history."
)


 ### Continue session: debug_session_id

User > Compare the trends of AAPL and MSFT based on their recent history.
enhanced_agent > Let's compare the trends of Apple (AAPL) and Microsoft (MSFT) over the last year.

**Apple (AAPL):**
The data indicates that Apple's stock has shown a **mixed trend** over the past year. It experienced an initial upward movement, reaching its peak around late October 2025. Following this, there was a notable decline in prices, with some recovery in the last few days of the provided data.

**Microsoft (MSFT):**
In contrast, Microsoft's stock has shown a **consistent downtrend** over the past year. While there were some fluctuations, the overall trajectory has been downwards, with the stock price lower at the end of the period compared to the beginning.

**Summary:**
Over the last year, AAPL has exhibited more volatility with periods of both gains and losses, while MSFT has been in a more definitive downtrend.


In [271]:
response = await runner.run_debug(
    "Is Amazon showing strong upward momentum recently?"
)


 ### Continue session: debug_session_id

User > Is Amazon showing strong upward momentum recently?
enhanced_agent > Analyzing Amazon's (AMZN) stock data for the last month, we can see a generally downward trend with significant volatility. While there were some positive movements, particularly around October 31st, the overall momentum appears to be weak, with prices declining towards the end of the period. Therefore, it is not showing strong upward momentum recently.


In [272]:
response = await runner.run_debug(
    "Show me the data you fetched to find the momentum."
)


 ### Continue session: debug_session_id

User > Show me the data you fetched to find the momentum.
enhanced_agent > Here is the data that was fetched for Amazon (AMZN) for the last month:

*   **2025-10-22:** Open: 219.30, High: 220.01, Low: 216.52, Close: 217.95
*   **2025-10-23:** Open: 219.00, High: 221.30, Low: 218.18, Close: 221.09
*   **2025-10-24:** Open: 221.97, High: 225.40, Low: 221.90, Close: 224.21
*   **2025-10-27:** Open: 227.66, High: 228.40, Low: 225.54, Close: 226.97
*   **2025-10-28:** Open: 228.22, High: 231.49, Low: 226.21, Close: 229.25
*   **2025-10-29:** Open: 231.67, High: 232.82, Low: 227.76, Close: 230.30
*   **2025-10-30:** Open: 227.06, High: 228.44, Low: 222.75, Close: 222.86
*   **2025-10-31:** Open: 250.10, High: 250.50, Low: 243.98, Close: 244.22
*   **2025-11-03:** Open: 255.36, High: 258.60, Low: 252.90, Close: 254.00
*   **2025-11-04:** Open: 250.38, High: 257.01, Low: 248.66, Close: 249.32
*   **2025-11-05:** Open: 249.03, High: 251.00, Low: 246.16,

In [276]:
response = await runner.run_debug(
    "Based on today’s market conditions, which sectors appear promising for long-term investors?"
)


 ### Continue session: debug_session_id

User > Based on today’s market conditions, which sectors appear promising for long-term investors?
enhanced_agent > I am sorry, but I cannot provide an analysis of "today's market conditions" or make predictions about which sectors appear promising. My capabilities are limited to fetching and analyzing historical stock data. I do not have access to real-time market sentiment, news, or the ability to predict future trends.


In [335]:
import yfinance as yf
from typing import Dict, Any

def get_stock_yahoo(symbol: str, period: str = "1mo") -> Dict[str, Any]:
    """
    Fetch real-time and recent historical stock data from Yahoo Finance.

    Args:
        symbol (str): Ticker symbol, e.g., "AAPL", "MSFT", "GOOG".
        period (str): Stock period of the Ticker, e.g., 1 day, 5 days, 1 month. By default, the value is "1mo"

    Returns:
        dict: {
            "status": "success",
            "data": {
                "symbol": str,
                "current_price": float,
                "history": list[dict],   # last 30 days OHLC
            }
        }
        OR, on failure:
        {
            "status": "error",
            "error_message": str
        }

    Notes:
        - Uses yfinance (unofficial Yahoo Finance API)
        - Provides last 1 month of daily historical data
        - Use inside ADK by adding to tools=[get_stock_yahoo]
    """
    try:
        ticker = yf.Ticker(symbol)

        # Fetch current price
        current_price = ticker.info.get("regularMarketPrice", None)

        if current_price is None:
            return {
                "status": "error",
                "error_message": f"Could not fetch current price for symbol '{symbol}'."
            }

        # Fetch last 1 month historical OHLC
        history_df = ticker.history(period=period)
        history_list = history_df.reset_index().to_dict(orient="records")

        return {
            "status": "success",
            "data": {
                "symbol": symbol,
                "current_price": current_price,
                "history": history_list
            }
        }

    except Exception as e:
        return {
            "status": "error",
            "error_message": f"Yahoo Finance request failed: {str(e)}"
        }


In [340]:
import yfinance as yf
from typing import Dict, Any

def get_stock_yahoo(symbol: str) -> Dict[str, Any]:
    """
    Fetch real-time and recent historical stock data from Yahoo Finance.

    Args:
        symbol (str): Ticker symbol, e.g., "AAPL", "MSFT", "GOOG".
        period (str): Data period such as "1d", "5d", "1mo", "3mo", "6mo", "1y", "5y", "ytd", "max".


    Returns:
        dict: {
            "status": "success",
            "data": {
                "symbol": str,
                "current_price": float,
                "history": list[dict],   # last 30 days OHLC
            }
        }
        OR, on failure:
        {
            "status": "error",
            "error_message": str
        }

    Notes:
        - Uses yfinance (unofficial Yahoo Finance API)
        - Provides last 1 month of daily historical data
        - Use inside ADK by adding to tools=[get_stock_yahoo]
    """
    try:
        ticker = yf.Ticker(symbol)

        # Fetch current price
        current_price = ticker.info.get("regularMarketPrice", None)

        if current_price is None:
            return {
                "status": "error",
                "error_message": f"Could not fetch current price for symbol '{symbol}'."
            }

        # Fetch last 1 month historical OHLC
        history_df = ticker.history(period="1mo")
        history_list = history_df.reset_index().to_dict(orient="records")

        return {
            "status": "success",
            "data": {
                "symbol": symbol,
                "current_price": current_price,
                "history": history_list
            }
        }

    except Exception as e:
        return {
            "status": "error",
            "error_message": f"Yahoo Finance request failed: {str(e)}"
        }


In [352]:
#from google_adk import tool
import yfinance as yf
from typing import Dict, Any

#@tool
def get_stock_yahoo(symbol: str, period: str = "1mo") -> Dict[str, Any]:
    """
    Fetch real-time and historical stock data from Yahoo Finance.

    Args:
        symbol (str): Stock ticker symbol.
        period (str): Data period such as "1d", "5d", "1mo", "3mo", "6mo", "1y", "5y", "ytd", "max".

    Returns:
        Success or error dictionary in ADK tool format.
    """
    try:
        ticker = yf.Ticker(symbol)
        current_price = ticker.info.get("regularMarketPrice")

        if current_price is None:
            return {
                "status": "error",
                "error_message": f"Unable to fetch price for {symbol}"
            }

        history_df = ticker.history(period=period)
        history_list = history_df.reset_index().to_dict(orient="records")

        return {
            "status": "success",
            "data": {
                "symbol": symbol,
                "current_price": current_price,
                "history": history_list
            }
        }

    except Exception as e:
        return {
            "status": "error",
            "error_message": str(e)
        }


In [374]:
#from google_adk import tool
import yfinance as yf
import pandas as pd
from typing import Dict, Any

#@tool
def get_stock_yahoo(symbol: str, period: str="1mo") -> Dict[str, Any]:
    """
    Fetch real-time and historical stock data from Yahoo Finance.
    Cleans NaN values and converts datetimes to JSON-safe strings.
    """

    try:
        ticker = yf.Ticker(symbol)

        current_price = ticker.info.get("regularMarketPrice")
        if current_price is None:
            return {
                "status": "error",
                "error_message": f"Unable to fetch price for {symbol}"
            }

        # Fetch history
        history_df = ticker.history(period=period)

        # Replace NaN with None
        history_df = history_df.replace({pd.NA: None, float("nan"): None})

        # Convert datetime index to string
        history_df.reset_index(inplace=True)
        history_df["Date"] = history_df["Date"].astype(str)

        # Convert to list of dictionaries
        history_list = history_df.to_dict(orient="records")

        return {
            "status": "success",
            "data": {
                "symbol": symbol,
                "current_price": current_price,
                "history": history_list
            }
        }

    except Exception as e:
        return {
            "status": "error",
            "error_message": str(e)
        }


In [388]:
import yfinance as yf
from typing import Dict, Any

def get_stock_yahoo(symbol: str, period: str="1mo") -> Dict[str, Any]:
    
    """
    Fetch real-time and historical stock data from Yahoo Finance.
    Cleans NaN values and converts datetimes to JSON-safe strings.

    Args:
        symbol (str): Ticker symbol, e.g., "AAPL", "MSFT", "GOOG".
        period (str): Stock period of the Ticker, e.g., 1 day, 5 days, 1 month. By default, the value is "1mo"

    Returns:
        dict: {
            "status": "success",
            "data": {
                "symbol": str,
                "current_price": float,
                "history": list[dict],   # last 30 days OHLC
            }
        }
        OR, on failure:
        {
            "status": "error",
            "error_message": str
        }

    Notes:
        - Uses yfinance (unofficial Yahoo Finance API)
        - Provides last 1 month of daily historical data
        - Use inside ADK by adding to tools=[get_stock_yahoo]
    """
    try:
        ticker = yf.Ticker(symbol)

        current_price = ticker.info.get("regularMarketPrice")
        if current_price is None:
            return {
                "status": "error",
                "error_message": f"Unable to fetch price for {symbol}"
            }

        # Fetch history
        history_df = ticker.history(period=period)

        # Replace NaN with None
        history_df = history_df.replace({pd.NA: None, float("nan"): None})

        # Convert datetime index to string
        history_df.reset_index(inplace=True)
        history_df["Date"] = history_df["Date"].astype(str)

        # Convert to list of dictionaries
        history_list = history_df.to_dict(orient="records")

        return {
            "status": "success",
            "data": {
                "symbol": symbol,
                "current_price": current_price,
                "history": history_list
            }
        }

    except Exception as e:
        return {
            "status": "error",
            "error_message": str(e)
        }

In [389]:
get_stock_yahoo("AAPL")

{'status': 'success',
 'data': {'symbol': 'AAPL',
  'current_price': 271.49,
  'history': [{'Date': '2025-10-22 00:00:00-04:00',
    'Open': 262.3956210979312,
    'High': 262.5954395959732,
    'Low': 255.18261234613632,
    'Close': 258.19970703125,
    'Volume': 45015300,
    'Dividends': 0.0,
    'Stock Splits': 0.0},
   {'Date': '2025-10-23 00:00:00-04:00',
    'Open': 259.68824995662277,
    'High': 260.36758405794774,
    'Low': 257.76012648314656,
    'Close': 259.3285827636719,
    'Volume': 32754900,
    'Dividends': 0.0,
    'Stock Splits': 0.0},
   {'Date': '2025-10-24 00:00:00-04:00',
    'Open': 260.93706434715574,
    'High': 263.87421967110856,
    'Low': 258.9290010880386,
    'Close': 262.56549072265625,
    'Volume': 38253700,
    'Dividends': 0.0,
    'Stock Splits': 0.0},
   {'Date': '2025-10-27 00:00:00-04:00',
    'Open': 264.6234656652487,
    'High': 268.8593494221352,
    'Low': 264.3936774471076,
    'Close': 268.5496520996094,
    'Volume': 44888200,
    'Di

In [399]:
agent = LlmAgent(
    name="enhanced_agent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=Http_options()),

    instruction="""
You are the Financial Stock Agent. You retrieve accurate stock data using tools and provide clear, safe, and useful financial insights. You must be informative and analytical without giving personalized financial advice.

───────────────────────────────────────────────
AUTOMATIC PERIOD HANDLING (IMPORTANT)
───────────────────────────────────────────────
When the user requests stock data with a time range, you MUST infer the period directly from the user's natural language.

Examples:
- “last year” → "1y"
- “last 12 months” → "1y"
- “last 5 years” → "5y"
- “past 3 months” → "3mo"
- “last week” → "5d"
- “today” → "1d"
- “YTD / year-to-date” → "ytd"

Never ask the user to confirm the period unless the question is ambiguous.  
Always call the Yahoo Finance tool with:  
get_stock_yahoo(symbol, period)

────────────────────────────────────────────────────────
YOUR ALLOWED BEHAVIOR
────────────────────────────────────────────────────────
You ARE allowed to:
- Identify promising sectors based on historical performance and current trends.
- Provide high-level investment insights.
- Explain market cycles and sector outlooks.
- Discuss long-term sector strength (e.g., technology, healthcare, semiconductors).
- Give probabilistic, research-based predictions and pattern interpretations.
- Use historical reasoning (“Historically…”, “Typically…", “This pattern suggests…”).
- Use google_search for sector, news, macroeconomic, and narrative insights.
- Call Yahoo Finance tool for real-time or historical stock numbers.

You are NOT allowed to:
- Give personalized financial advice.
- Guarantee profits or future outcomes.
- Tell users what they “should” invest in.
- Fabricate numbers not retrieved from the Yahoo Finance tool.

────────────────────────────────────────────────────────
CORE RESPONSIBILITIES
────────────────────────────────────────────────────────
1. Use Yahoo Finance tool for:
   - Current stock price
   - Historical prices
   - Trend analysis
   - Volatility interpretation
   - Performance explanations  
   Default history period: 1 month  
   Adjust this based on user request.

2. Use google_search for:
   - Sector performance insights
   - Industry-level news and trends
   - Economic context
   - Company background
   - Analyst commentary (summaries)
   - Growth trend explanations

3. Provide:
   - Clearly with Numerical explanations when stock data is available
   - Historical pattern interpretation
   - Sector rotation insights
   - Long-term macro reasoning
   - Risk and opportunity analysis

4. Prediction-Style Reasoning:
   - Must be probabilistic, never guaranteed.
   - Use phrases like:
       “Based on historical data…”
       “This pattern usually indicates…”
       “Analysts generally interpret this as…”
       “This may suggest…”

────────────────────────────────────────────────────────
TOOL USAGE RULES
────────────────────────────────────────────────────────
You MUST call:
- Yahoo Finance tool **whenever specific stock data is needed**
- google_search **whenever user asks sector-level, industry-level, or news-level questions**

Never fabricate financial numbers.

────────────────────────────────────────────────────────
TONE & STYLE
────────────────────────────────────────────────────────
Be:
- Professional
- Clear
- Insightful
- Educational
- Data-driven

Your goal:
- Give strong insights (with Numerical explanation - wherever needed) while remaining safe and non-prescriptive.
""",

    tools=[get_stock_yahoo,],
)


In [400]:
runner = InMemoryRunner(agent=agent)
response = await runner.run_debug(
    "Fetch Microsoft’s stock data for the last year and tell me if it is in an uptrend or downtrend and suggest me if it is good to invest at this time."
)


 ### Created new session: debug_session_id

User > Fetch Microsoft’s stock data for the last year and tell me if it is in an uptrend or downtrend and suggest me if it is good to invest at this time.
enhanced_agent > Based on the stock data from Yahoo Finance for Microsoft (MSFT) over the past year:

**Trend Analysis:**
Microsoft's stock has shown a general **uptrend** over the last year. While there have been fluctuations and periods of decline, the overall trajectory of the closing prices indicates an upward movement.

**Investment Suggestion:**
As an AI financial agent, I cannot provide personalized investment advice or recommendations. Whether it is a good time to invest in Microsoft's stock depends on various factors, including your individual financial goals, risk tolerance, and investment horizon.

To make an informed decision, I recommend:
*   **Further Research:** Look into recent company news, earnings reports, and analyst ratings for Microsoft.
*   **Market Conditions:** Con

In [396]:
response = await runner.run_debug(
    "Compare the trends of AAPL and MSFT based on their recent history."
)


 ### Continue session: debug_session_id

User > Compare the trends of AAPL and MSFT based on their recent history.
enhanced_agent > Microsoft's stock exhibited a general upward trend over the past year, with some fluctuations. Its performance indicates a positive trajectory.

Apple's stock also showed an upward trend over the last year, although it experienced more volatility compared to Microsoft. Despite the fluctuations, the overall movement has been positive.

In summary, both stocks have demonstrated positive performance over the past year. Microsoft's trend appears to be more consistently upward, while Apple's trend, though also positive, has been more volatile.


In [379]:
response = await runner.run_debug(
    "Is Amazon showing strong upward momentum recently?"
)


 ### Continue session: debug_session_id

User > Is Amazon showing strong upward momentum recently?
enhanced_agent > Yes, Amazon (AMZN) has shown strong upward momentum recently.

Here's a look at its performance over the last month:

*   **Start of the month (Oct 22, 2025):** The stock opened at approximately $219.30.
*   **End of the month (Nov 21, 2025):** The stock closed at around $220.69.

While the closing price at the very end of the period isn't drastically higher than the opening, the data shows significant positive movement within the month. Key indicators of this upward momentum include:

*   **Increase from lows:** The stock reached a low of $215.18 on November 21st, but traded as high as $258.60 on November 3rd.
*   **Price Recovery:** After a dip to $222.86 on October 30th, the stock recovered and reached new highs, indicating buying interest and positive sentiment.

This pattern of recovery and reaching higher price points suggests that Amazon's stock has recently exper

In [380]:
response = await runner.run_debug(
    "Show me the data you fetched to find the momentum."
)


 ### Continue session: debug_session_id

User > Show me the data you fetched to find the momentum.
enhanced_agent > Here is the historical stock data for Amazon (AMZN) over the last month:

*   **Date:** 2025-10-22, **Close:** 217.95
*   **Date:** 2025-10-23, **Close:** 221.09
*   **Date:** 2025-10-24, **Close:** 224.21
*   **Date:** 2025-10-27, **Close:** 226.97
*   **Date:** 2025-10-28, **Close:** 229.25
*   **Date:** 2025-10-29, **Close:** 230.30
*   **Date:** 2025-10-30, **Close:** 222.86
*   **Date:** 2025-10-31, **Close:** 244.22
*   **Date:** 2025-11-03, **Close:** 254.00
*   **Date:** 2025-11-04, **Close:** 249.32
*   **Date:** 2025-11-05, **Close:** 250.20
*   **Date:** 2025-11-06, **Close:** 243.04
*   **Date:** 2025-11-07, **Close:** 244.41
*   **Date:** 2025-11-10, **Close:** 248.40
*   **Date:** 2025-11-11, **Close:** 249.10
*   **Date:** 2025-11-12, **Close:** 244.20
*   **Date:** 2025-11-13, **Close:** 237.58
*   **Date:** 2025-11-14, **Close:** 234.69
*   **Date:** 202

In [381]:
runner = InMemoryRunner(agent=agent)
response = await runner.run_debug(
    "Based on today’s market conditions, which sectors appear promising for long-term investors?"
)


 ### Created new session: debug_session_id

User > Based on today’s market conditions, which sectors appear promising for long-term investors?
enhanced_agent > Here's an analysis of the sectors based on their year-to-date performance:

**Technology (XLK):**
*   Year-to-date, the Technology sector has shown a positive trend, indicating strong investor interest and potential for growth. This sector often benefits from innovation and demand for new digital solutions.

**Healthcare (XLV):**
*   The Healthcare sector has also exhibited positive performance year-to-date. This sector is generally considered defensive, meaning it tends to be more stable during economic downturns, and is driven by consistent demand for health services and products.

**Financials (XLF):**
*   The Financial sector has shown mixed performance year-to-date. While there have been periods of growth, it also appears to be sensitive to broader economic conditions and interest rate changes.

**Utilities (XLU):**
*   Th

In [ ]:
# tools/agent_scheduler_tool.py
import json
import os
from typing import Dict, Any
from datetime import datetime
import yfinance as yf

WATCHLIST_FILE = "stock_watchlist.json"

def _load_watchlist():
    if not os.path.exists(WATCHLIST_FILE):
        return {"watchlist": []}
    with open(WATCHLIST_FILE, "r") as f:
        return json.load(f)

def _save_watchlist(data):
    with open(WATCHLIST_FILE, "w") as f:
        json.dump(data, f, indent=4)


def add_stock_alert(symbol: str, threshold: float, direction: str) -> Dict[str, Any]:
    """
    Add a stock alert to the monitoring list.

    Args:
        symbol (str): Stock ticker.
        threshold (float): Price threshold.
        direction (str): "above" or "below".

    Returns:
        dict: ADK tool-standard success or error dictionary.
    """
    try:
        data = _load_watchlist()

        data["watchlist"].append({
            "symbol": symbol.upper(),
            "threshold": threshold,
            "direction": direction.lower(),
            "last_notified": None
        })

        _save_watchlist(data)

        return {
            "status": "success",
            "data": f"Alert added: Notify when {symbol.upper()} goes {direction} {threshold}"
        }

    except Exception as e:
        return {"status": "error", "error_message": str(e)}



def check_alerts() -> Dict[str, Any]:
    """
    Checks all stored stock alerts.
    Returns triggered notifications.

    This tool is intended to be called by your scheduler.
    """
    try:
        data = _load_watchlist()
        notifications = []

        for item in data["watchlist"]:
            ticker = yf.Ticker(item["symbol"])
            price = ticker.info.get("regularMarketPrice")

            if price is None:
                continue

            condition_met = (
                (item["direction"] == "above" and price >= item["threshold"]) or
                (item["direction"] == "below" and price <= item["threshold"])
            )

            if condition_met:
                notifications.append({
                    "symbol": item["symbol"],
                    "price": price,
                    "threshold": item["threshold"],
                    "message": f"{item['symbol']} is now {price}, which is {item['direction']} {item['threshold']}"
                })

                # update last notified timestamp
                item["last_notified"] = str(datetime.now())

        _save_watchlist(data)

        return {
            "status": "success",
            "data": notifications
        }

    except Exception as e:
        return {"status": "error", "error_message": str(e)}


In [ ]:
# src/scheduler.py
import time
import subprocess
from monitor_tools import check_alerts

CHECK_INTERVAL = 60  # seconds

def run_scheduler():
    print("Stock Monitoring Scheduler Started.")
    
    while True:
        print("Checking alerts...")
        result = check_alerts()

        if result["status"] == "success" and result["data"]:
            print("\n=== ALERTS TRIGGERED ===")
            for alert in result["data"]:
                print(alert["message"])
                # TODO: send SMS/email/discord, etc
            print("========================\n")

        time.sleep(CHECK_INTERVAL)

if __name__ == "__main__":
    run_scheduler()


In [ ]:
# src/scheduler_launcher.py
import subprocess
import psutil
from typing import Dict, Any

SCHEDULER_NAME = "scheduler.py"

def is_scheduler_running() -> bool:
    for proc in psutil.process_iter(["pid", "cmdline"]):
        try:
            if proc.info["cmdline"] and SCHEDULER_NAME in proc.info["cmdline"]:
                return True
        except:
            pass
    return False

def launch_scheduler() -> Dict[str, Any]:
    try:
        subprocess.Popen(
            ["python", SCHEDULER_NAME],
            stdout=subprocess.DEVNULL,
            stderr=subprocess.STDOUT
        )
        return {"status": "success", "data": "Scheduler started successfully."}

    except Exception as e:
        return {"status": "error", "error_message": str(e)}


In [ ]:
# tools/agent_scheduler_tool.py
from typing import Dict, Any
from scheduler_launcher import is_scheduler_running, launch_scheduler

def ensure_scheduler_running() -> Dict[str, Any]:
    """
    Ensures the stock monitoring scheduler is running.
    Called automatically before creating a new stock alert.
    """
    # If already running → do nothing
    if is_scheduler_running():
        return {
            "status": "success",
            "data": "Scheduler already running."
        }

    # Otherwise launch it
    return launch_scheduler()


In [ ]:
agent = LlmAgent(
    name="enhanced_agent",
    model=Gemini(model="gemini-2.5-flash-lite"),
    instruction="...",   # your main instructions
    tools=[
        get_stock_yahoo,
        google_search,
        add_stock_alert,
        ensure_scheduler_running  # this will auto-start scheduler
    ]
)


In [10]:
os.getcwd()

'c:\\PG\\Projects'

In [ ]:
%cd ..

In [19]:
from tools.Yahoo_Finance_Function_tool import get_stock_yahoo
from tools.stock_monitor_tool import StockMonitorTool
from google.adk.agents import LlmAgent
from src.helper import Helper

# ─────────────────────────────────────────────
# Instantiate Monitor Tool (merged class)
# ─────────────────────────────────────────────
monitor = StockMonitorTool()


# ─────────────────────────────────────────────
# FINANCE AGENT SETUP
# ─────────────────────────────────────────────
agent = LlmAgent(
    name="enhanced_agent",

    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=Helper.Http_options()
    ),

    # Main instructions
    instruction="""
You are the Financial Stock Agent. Your mission is to fetch real stock data,
analyze trends, interpret sector conditions, and provide safe, data-driven insights.

You MUST use tools for:
- Yahoo Finance data
- Google Search for company/sector/news context
- Stock monitoring tools

NEVER fabricate numbers.
NEVER give guaranteed financial advice.
All predictions must be probabilistic, historically grounded, and safe.

When the user asks to set an alert:
1. Ensure scheduler is running
2. Add the alert to the watchlist

When the user asks for stock performance:
• Call the Yahoo Finance tool
• Then analyze the returned data

When the user asks about sectors, news, trends:
• Use google_search when external information is required

Maintain a professional, clear, and analytical tone.
""",

    # Tools exposed to the agent
    tools=[
        get_stock_yahoo,                 # Real stock data
        google_search,                   # Sector/company/news context

        # Monitoring tools (methods of the merged class)
        monitor.add_stock_alert,
        monitor.check_alerts,
        monitor.ensure_scheduler_running
    ]
)

In [ ]:
from tools.stock_monitor_tool import StockMonitorTool
from src.helper import Helper

# ─────────────────────────────────────────────
# Instantiate the Stock Monitor
# ─────────────────────────────────────────────
monitor = StockMonitorTool()


# ─────────────────────────────────────────────
# Monitoring Agent (Scheduler auto-starts inside add_stock_alert)
# ─────────────────────────────────────────────
monitoring_agent = LlmAgent(
    name="monitoring_agent",

    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=Helper.Http_options()
    ),

    instruction="""
You are the Stock Monitoring Agent.

Your ONLY job is to use the monitoring tools to:
1. Add stock alerts
2. Check alerts
3. Ensure the scheduler is running (if user explicitly asks)

────────────────────────────────────────
TOOL CALLING RULES (IMPORTANT)
────────────────────────────────────────

You MUST follow these exact tool signatures:

1. To add a stock alert:
   add_stock_alert(
       symbol=str,
       threshold=float,
       direction=str  # 'above' or 'below'
   )

2. To check triggered alerts:
   check_alerts()

3. To ensure the scheduler is active:
   ensure_scheduler_running()

DO NOT add extra arguments.
DO NOT rename arguments.
DO NOT create new parameters.
DO NOT send dicts or objects.
ONLY use the exact arguments in the signature.

Example VALID tool calls:
- add_stock_alert(symbol="AAPL", threshold=150, direction="below")
- add_stock_alert(symbol="TSLA", threshold=300, direction="above")
- check_alerts()
- ensure_scheduler_running()

Example INVALID calls (NEVER do these):
- add_stock_alert(ticker="AAPL", price=150, dir="below")
- add_stock_alert({"symbol":"AAPL", "threshold":150, "direction":"below"})
- check_alerts(args={})
- ensure_scheduler_running("start")

────────────────────────────────────────
AGENT BEHAVIOR
────────────────────────────────────────

When user asks:
• "alert me when AAPL goes below 150"
→ Call add_stock_alert(symbol="AAPL", threshold=150, direction="below")

• "monitor TSLA above 300"
→ Call add_stock_alert(symbol="TSLA", threshold=300, direction="above")

• "check alerts"
→ Call check_alerts()

• "show notifications"
→ Call check_alerts()

• "start scheduler" or "is scheduler running?"
→ Call ensure_scheduler_running()

DO NOT provide financial analysis.
DO NOT pull stock prices.
DO NOT use google_search or get_stock_yahoo.
You ONLY manage alerts and scheduler behavior.

Keep responses short and functional.
""",

    # Tools exposed to the agent
    tools=[
        monitor.add_stock_alert,          # Auto-starts scheduler internally
        monitor.check_alerts,             # Scheduler will call this too
        monitor.ensure_scheduler_running  # Exposed only for explicit user requests
    ]
)


In [25]:
runner = InMemoryRunner(agent=agent)
response = await runner.run_debug(
    "Alert me when AAPL falls below 150"
)

App name mismatch detected. The runner is configured with app name "InMemoryRunner", but the root agent was loaded from "C:\PG\Projects\Personalized-Finance-Agent\venv\Lib\site-packages\google\adk\agents", which implies app name "agents".
Tools at indices [0] are not compatible with automatic function calling (AFC). AFC is disabled. If AFC is intended, please include python callables in the tool list, and do not include function declaration in the tool list.



 ### Created new session: debug_session_id

User > Alert me when AAPL falls below 150


ClientError: 400 INVALID_ARGUMENT. {'error': {'code': 400, 'message': 'Tool use with function calling is unsupported', 'status': 'INVALID_ARGUMENT'}}